<a href="https://colab.research.google.com/github/cmencar/descriptive_stability/blob/develop/descriptive_stability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook implements the method published in:

> C. Mencar, C. Castiello, *Descriptive Stability of Fuzzy Rule-Based Systems*, proc. of FuzzIEEE2021 -- IEEE International Conf. on Fuzzy Systems 2021 (in press)

The code is released under [CC BY-NC-SA](https://creativecommons.org/licenses/by-nc-sa/4.0/); This license lets others remix, adapt, and build upon your work non-commercially, as long as they credit the authors and license their new creations under the identical terms.  

# Preliminaries

## Required modules and functions

In [1]:
from collections import namedtuple
from math import exp
from scipy.optimize import linear_sum_assignment
from itertools import combinations
import numpy as np

## Utility functions

In [2]:
χ = lambda x,y: 1 if x==y else 0

### Aggregation functions

In [3]:
def prod(*it):
  """Product of an iterable"""
  p = 1
  for elem in it:
    p *= elem
  return p

In [4]:
def owa(w, *it):
  """Ordered Weighted Average of an iterable"""
  sorted_it = sorted(it)
  r = sum(elem*weight for elem,weight in zip(sorted_it, w))
  #print(f"aggregation of {it} is: {r}")
  return r

In [5]:
avg = lambda *it: owa([1/len(it)]*len(it), *it)

### Set functions

In [6]:
def jaccard(s1, s2):
  return len(s1 & s2) / len(s1 | s2)

### Fuzzy set functions

#### Membership functions

In [7]:
def trapmf(a,b,c,d):
  return lambda x:\
            (x-a)/(b-a) if a < x < b else\
            1           if b <= x <= c else\
            (x-d)/(c-d) if c < x < d else\
            0

In [8]:
trimf = lambda a,b,c: trapmf(a,b,b,c)

In [9]:
def singletonmf(v, delta):
  return lambda x:\
            1 if v-delta <= x < v+delta else\
            0

#### Fuzzy set manipulation

In [10]:
ext_mf = lambda mf, U: (lambda x: 0 if x not in U else mf(x))

In [11]:
σ_count = lambda mf, U: sum(mf(x) for x in U)

# FRBS structure

In [12]:
FRBS = namedtuple("FRBS", ["DB", "RB"])

In [13]:
LV = namedtuple("LV", ["X", "T", "U", "G", "μ"])
# DB is a dictionary of {X:LV}

In [14]:
Rule = namedtuple("Rule", ["A", "C"])
# RB is a dictionary of {str:Rule}

In [15]:
Consequent = namedtuple('Consequent', ['Y', 'w', 'ctype'])

In [16]:
SoftConstraint = namedtuple("SoftCostraint", ["X", "t"])
# A is a set of SoftConstraint

# Descriptive Stability

## Common settings

In [18]:
𝒶 = avg  # Aggregation function

In [19]:
σ = jaccard  # Set similarity

## Rule Base Similarity

### Rule Similarity

#### Consequent Similarity

In [20]:
def CS(C1,C2):
  """Consequent Similarity"""
  assert C1.ctype == C2.ctype

  if C1.ctype == "classification":
    r = prod(χ(C1.Y, C2.Y),χ(C1.w, C2.w))
  elif C1.ctype == "mamdani":
    r = prod(χ(C1.Y, C2.Y),χ(C1.w, C2.w))
  elif C1.ctype == "tsk0":
    r = prod(χ(C1.Y, C2.Y), exp(-(C1.w - C2.w)**2))   
  else:
    raise NotImplementedError

  #print(f"CS({C1},{C2})={r}")  
  return r

#### Antecedent Similarity

In [21]:
ASN = lambda A: {SC.X for SC in A}

In [22]:
def AS(A1, A2):
  r = 𝒶(σ(ASN(A1), ASN(A2)), *[χ(SC1.t, SC2.t) for SC1 in A1 for SC2 in A2 if SC1.X==SC2.X])
  #print(f"AS({A1}, {A2})={r}")
  return r

#### Rule Similarity

### Rule Base Similarity

In [23]:
def RBASS(RB1, RB2):
  M = np.array([RS(r1,r2) for r1 in RB1.values() for r2 in RB2.values()])\
              .reshape(len(RB1),len(RB2))
  row_ind, col_ind = linear_sum_assignment(M, maximize=True)
  r = M[row_ind,col_ind].sum()
  #print(f"RBASS({RB1}, {RB2})={r}")
  return r

  

In [24]:
def RBS(RB1, RB2):
  r1 = len(RB1)
  r2 = len(RB2)
  r = RBASS(RB1, RB2)/max(r1,r2)
  #print(f"RBS({RB1}, {RB2})={r}")
  return r

In [25]:
def RS(R1, R2):
  r = 𝒶(AS(R1.A, R2.A), CS(R1.C, R2.C))
  #print(f"RS({R1}, {R2})={r}")
  return r

## Data Base Similarity

In [26]:
DBN = lambda DB: {LV.X for LV in DB.values()}

### Data Base Interpretation Similarity

#### Interpretation Similarity

In [27]:
def IS(μ1, μ2, U1, U2):
  U_min = U1 & U2
  U_max = U1 | U2
  min_mf = lambda x: min(ext_mf(μ1, U_min)(x), ext_mf(μ2, U_min)(x))
  max_mf = lambda x: max(ext_mf(μ1, U_max)(x), ext_mf(μ2, U_max)(x))
  r = σ_count(min_mf, U_min) / σ_count(max_mf, U_max)
  #print(f"IS({μ1}, {μ2}, {U1}, {U2})={r}")
  return r

#### Data Base Interpretation Similarity

In [28]:
def DBIS(DB1, DB2):
  r = 𝒶(*[IS(LV1.μ(t), LV2.μ(t), LV1.U, LV2.U)
              for LV1 in DB1.values()
              for LV2 in DB2.values()
              for t in LV1.T & LV2.T
              if LV1.X == LV2.X])
  #print(f"DBIS({DB1}, {DB2})={r}")
  return r


### Data Base Term Similarity

#### Term Similarity

In [29]:
def TS(T1, T2):
  r = σ(T1, T2)
  #print(f"TS({T1}, {T2})={r}")  
  return r

#### Data Base Term Similarity

In [30]:
def DBTS(DB1, DB2):
  r = 𝒶(*[
      TS(LV1.T, LV2.T)
      for LV1 in DB1.values()
      for LV2 in DB2.values()
      if LV1.X == LV2.X
  ])
  #print(f"DBTS({DB1}, {DB2})={r}")
  return r

### Data Base Name Similarity

In [31]:
def DBNS(DB1, DB2):
  r = σ(DBN(DB1), DBN(DB2))
  #print(f"DBNS({DB1}, {DB2}) = {r}")  
  return r

### Data Base Similarity

In [32]:
def DBS(DB1, DB2):
  #print("DEBUG: FORCED DBS=1"); r=1
  #r = DBNS(DB1, DB2) * 𝒶(DBTS(DB1, DB2), DBIS(DB1, DB2)) # aggregation applies to the proportion of LVs in common
  r = 𝒶(DBNS(DB1, DB2), DBTS(DB1, DB2), DBIS(DB1, DB2)) # aggregation applies to the proportion of LVs in common
  #print(f"DBS({DB1}, {DB2}) = {r}")
  return r 

## FRBS Similarity

In [33]:
def KBS(S1, S2):
  r = 𝒶(DBS(S1.DB, S2.DB), RBS(S1.RB, S2.RB))
  #print(f"KBS({S1}, {S2})={r}")
  return r

In [34]:
def DS(*S):
  r = 𝒶(*[KBS(*S_pair) for S_pair in combinations(S,2)])
  #print(f"DS({S})={r}")
  return r

# FML file parsing

In [35]:
import xml.etree.ElementTree as ET

In [36]:
def buildfromFML(filename, fsresolution=100):
    xmlns = "{http://www.ieee1855.org}"
    tree = ET.parse(filename)
    FRBS_XML = tree.getroot()
    DB_XML =  FRBS_XML[0]
    RB_XML =  FRBS_XML[1]

    # processing DB_XML
    DB = {}
    for LV_XML in DB_XML:
        X = LV_XML.attrib['name']
        U_left = float(LV_XML.attrib['domainleft'])
        U_right = float(LV_XML.attrib['domainright'])
        U = set(np.linspace(U_left, U_right, fsresolution))
        delta = (U_right-U_left) / fsresolution
        G = {}
        T = set()
        μ_dict = {}
        for LT_XML in LV_XML:
            t = LT_XML.attrib['name']
            T |= {t}
            MF_XML = LT_XML[0]
            if MF_XML.tag == xmlns+'trapezoidShape':
                a = float(MF_XML.attrib['param1'])
                b = float(MF_XML.attrib['param2'])
                c = float(MF_XML.attrib['param3'])
                d = float(MF_XML.attrib['param4'])
                mf = trapmf(a, b, c, d)
            elif MF_XML.tag == xmlns+'triangularShape':
                a = float(MF_XML.attrib['param1'])
                b = float(MF_XML.attrib['param2'])
                c = float(MF_XML.attrib['param3'])
                mf = trimf(a, b, c)
            elif MF_XML.tag == xmlns+'singletonShape':
                v = float(MF_XML.attrib['param1'])
                mf = singletonmf(v, delta)
            else:
                raise NotImplementedError
            
            μ_dict[t] = mf
        if len(T) > 0:  # discard LVs without terms
          μ = μ_dict.get
          DB[X] = LV(X,T,U,G,μ)

    # processing RB_XML
    if RB_XML.tag == xmlns+'mamdaniRuleBase':
        ctype = 'mamdani'
    else:
        raise NotImplementedError

    RB = {}
    for RULE_XML in RB_XML:
        ANT_XML = RULE_XML[0]
        A = set()
        for SC_XML in ANT_XML:
            X = SC_XML[0].text
            t = SC_XML[1].text
            A |= {SoftConstraint(X,t)}
            
        CONS_XML = RULE_XML[1][0][0]
        Y = CONS_XML[0].text
        w = CONS_XML[1].text
        C = Consequent(Y, w, ctype)
        
        RB[RULE_XML.attrib['name']] = Rule(A,C)


    return FRBS(DB, RB)

## Test

In [37]:
'''
Use in COLAB only
'''
# from google.colab import drive
# drive.mount('/content/drive')

In [38]:
path_locale = './DS-experiments/'
path_gdrive = '/content/drive/MyDrive/DS-experiments/'
path = path_locale

In [39]:
#frbs1 = buildfromFML(path+'InvertedPendulumMamdani1.xml')
#frbs2 = buildfromFML(path+'InvertedPendulumMamdani1.xml')

In [40]:
#print(DS(frbs1, frbs2))

# Experiments

In [41]:
def do_experiment(folder, filename_template, it=range(10)):
  try:
    frbss = [buildfromFML(path+folder+'/'+filename_template.format(i)) for i in it]
    ds_global = DS(*frbss)
    ds_matrix = np.array([[DS(fs1, fs2) for fs1 in frbss] for fs2 in frbss])
    return ds_global, ds_matrix, frbss
  except:
    print(f"Failed on: {folder}/{filename_template}")
    raise

## Run experiments

In [42]:
folders = ["FDT_Beer", 
           "beer-guaje-fdtp-s-jfml", 
           "BEER-guaje-furia-jfml",
           "PIMA-guaje-furia-jfml",
           "PIMA-guaje-furia-jfml",
           "PIMA-guaje-furia-jfml",
           "WINE-guaje-furia-jfml",
           "WINE-guaje-furia-jfml",
           "WINE-guaje-furia-jfml"]
filename_templates = ["cv{0}-guaje-fdtp.jfml.xml", 
                      'cv{0}-guaje-fdtp-s.jfml.xml', 
                      'BEER3.txt.aux.train.{0}.arff.FURIA.log.txt.rules.txt.jfml.xml',
                      "cv{0}-guaje-fdtp.jfml.xml",
                      "cv{0}-guaje-fdtp-s.jfml.xml",
                      "PIMA.txt.aux.train.{0}.arff.FURIA.log.txt.rules.txt.jfml.xml",
                      "cv{0}-guaje-fdtp.jfml.xml",
                      "cv{0}-guaje-fdtp-s.jfml.xml",
                      "WINE.txt.aux.train.{0}.arff.FURIA.log.txt.rules.txt.jfml.xml"
]
experiments = {folder+'/'+filename_template: do_experiment(folder, filename_template) 
                for folder, filename_template in zip(folders, filename_templates)}

## Print results

In [43]:
np.set_printoptions(precision=3)
for exp in experiments:
  print("EXPERIMENT: ", exp)
  print("Global DS: ", experiments[exp][0])
  print("Pairwise comparison: ")
  print(experiments[exp][1])
  print()

EXPERIMENT:  FDT_Beer/cv{0}-guaje-fdtp.jfml.xml
Global DS:  0.9018448681625427
Pairwise comparison: 
[[1.    0.771 0.779 0.777 0.751 0.781 0.777 0.78  0.781 0.736]
 [0.771 1.    0.988 0.985 0.795 0.986 0.984 0.989 0.984 0.926]
 [0.779 0.988 1.    0.996 0.794 0.997 0.995 0.999 0.996 0.929]
 [0.777 0.985 0.996 1.    0.792 0.994 0.995 0.995 0.993 0.932]
 [0.751 0.795 0.794 0.792 1.    0.797 0.791 0.794 0.794 0.752]
 [0.781 0.986 0.997 0.994 0.797 1.    0.992 0.996 0.996 0.928]
 [0.777 0.984 0.995 0.995 0.791 0.992 1.    0.994 0.992 0.929]
 [0.78  0.989 0.999 0.995 0.794 0.996 0.994 1.    0.995 0.93 ]
 [0.781 0.984 0.996 0.993 0.794 0.996 0.992 0.995 1.    0.928]
 [0.736 0.926 0.929 0.932 0.752 0.928 0.929 0.93  0.928 1.   ]]

EXPERIMENT:  beer-guaje-fdtp-s-jfml/cv{0}-guaje-fdtp-s.jfml.xml
Global DS:  0.8498221149201389
Pairwise comparison: 
[[1.    0.668 0.659 0.658 0.724 0.659 0.658 0.66  0.659 0.684]
 [0.668 1.    0.946 0.945 0.765 0.945 0.945 0.946 0.943 0.832]
 [0.659 0.946 1.    0.99

#### cross-comparison

In [44]:
ds1 = "FDT_Beer/cv{0}-guaje-fdtp.jfml.xml"
ds2 = "beer-guaje-fdtp-s-jfml/cv{0}-guaje-fdtp-s.jfml.xml"

comparison = np.array([[DS(fs1, fs2) for fs1 in experiments[ds1][2]] for fs2 in experiments[ds2][2]])

In [45]:
print(ds1, "vs.", ds2); print(comparison)

FDT_Beer/cv{0}-guaje-fdtp.jfml.xml vs. beer-guaje-fdtp-s-jfml/cv{0}-guaje-fdtp-s.jfml.xml
[[0.642 0.632 0.632 0.631 0.543 0.632 0.631 0.632 0.632 0.611]
 [0.547 0.624 0.623 0.621 0.547 0.622 0.622 0.624 0.62  0.586]
 [0.527 0.599 0.6   0.598 0.531 0.598 0.598 0.599 0.596 0.562]
 [0.526 0.597 0.598 0.6   0.531 0.597 0.597 0.597 0.595 0.562]
 [0.619 0.711 0.711 0.711 0.622 0.712 0.71  0.711 0.709 0.682]
 [0.527 0.598 0.598 0.597 0.533 0.6   0.597 0.597 0.595 0.561]
 [0.526 0.598 0.598 0.597 0.53  0.597 0.6   0.598 0.597 0.562]
 [0.527 0.599 0.599 0.597 0.531 0.597 0.598 0.6   0.596 0.562]
 [0.527 0.596 0.596 0.595 0.529 0.595 0.597 0.596 0.6   0.561]
 [0.534 0.597 0.593 0.593 0.522 0.593 0.593 0.593 0.592 0.637]]


In [46]:
comparison.diagonal().mean()

0.6122916778735743

In [47]:
comparison.diagonal().std()

0.016156192942621148